## Set-up Joint Distribution

In [5]:
import tensorflow_probability as tfp
tfd = tfp.distributions
import tensorflow as tf
# disabling gpu for now
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [15]:
n = tfp.distributions.Sample(tfp.distributions.Normal(1, 1) + tfp.distributions.Normal(1, 1), [1,10])

TypeError: unsupported operand type(s) for +: 'Normal' and 'Normal'

In [ ]:
L = 10
seq = [tfd.Independent(tfd.Sample(tfd.TruncatedNormal(loc=0.2, scale=0.1, low=0.01, high=1.0), 10))]
for i in range(L - 1):
    seq.append(tfd.Sample(tfd.TruncatedNormal(loc=0.2, scale=0.1, low=0.01, high=1.0))
column = tfd.JointDistributionSequential([
        ,
        tfd.Uniform(0.01, 0.25)])

In [13]:
p = tfd.Sample(tfd.TruncatedNormal(loc=0.2, scale=0.1, low=0.01, high=1.0), [5,5])
tp = tfd.TransformedDistribution(p, tfp.bijectors.Cumsum(axis=-1))
tp.sample()

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0.28826424, 0.55218494, 0.79044604, 0.8499169 , 0.9590945 ],
       [0.2804437 , 0.48286095, 0.6556516 , 1.11135   , 1.3382541 ],
       [0.22602803, 0.5312986 , 0.72557545, 0.79157126, 0.9535962 ],
       [0.22573346, 0.60500956, 0.80750585, 0.89141303, 1.2722808 ],
       [0.11767041, 0.30089685, 0.51801586, 0.74979913, 1.04429   ]],
      dtype=float32)>

In [16]:
joint = tfd.JointDistributionSequential([
        tfd.Independent(tfd.Sample(tfd.TruncatedNormal(loc=0.2, scale=0.1, low=0.01, high=1.0), [5,5])),
        tfd.Uniform(0.01, 0.25)])

In [17]:
joint.sample()

[<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
 array([[0.18558306, 0.04564987, 0.32678637, 0.47217745, 0.16273353],
        [0.29458594, 0.2068963 , 0.270828  , 0.03753713, 0.13953906],
        [0.23399927, 0.08555575, 0.12613174, 0.24499345, 0.08525956],
        [0.23260525, 0.27436814, 0.12040873, 0.31305236, 0.24820961],
        [0.14738834, 0.14449677, 0.37170205, 0.15917276, 0.09838676]],
       dtype=float32)>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.24376105>]